### Set TACC account credentials for authenticating to the Tapis TACC Tenant 

In [22]:
import getpass
permitted_username = 'testuser2'
permitted_user_password = 'testuser2'
# permitted_username = getpass.getpass(prompt='Username: ', stream=None)
# permitted_user_password = getpass.getpass(prompt='Password: ', stream=None)

#Set Tapis Tenant and Base URL
tenant="dev"
base_url = 'https://' + tenant + '.develop.tapis.io'

#Load Python SDK
from tapipy.tapis import Tapis

#Create python Tapis client for user
permitted_client = Tapis(base_url= base_url, 
                         username=permitted_username,
                         password=permitted_user_password, 
                         account_type='user', 
                         tenant_id=tenant
                        ) 


#Generate an Access Token that will be used for all API calls
permitted_client.get_tokens()

permitted_client.access_token

BadRequestError: message: Invalid username/password combination.

### Listing All Existing Projects
- To check if streams api is working correctly

In [ ]:
permitted_client.streams.list_projects()

### Setup Streams Variables that are used in the rest of the notebook

In [10]:
import datetime
# storage_id = "dev.tapisv3.storage.system"
project_id = 'Mesonet' + str(datetime.datetime.today().isoformat())
site_id = 'ParkHQ'
instrument_id = '0151_ParkHQ' + str(datetime.datetime.today().isoformat()).replace(".", "-").replace(":", "-")

In [14]:
instrument_id = '0151_ParkHQ' + str(datetime.datetime.today().isoformat()).replace(".", "-").replace(":", "-")

### Creating the MesoNet Project

In [11]:
result = permitted_client.streams.create_project(project_name=project_id,
                                                        description='TEST project for MesoNet',
                                                        owner='testuser2', pi='wongy', 
                                                        funding_resource='test', 
                                                        project_url='https://www.hawaii.edu/climate-data-portal/',
                                                        active=True,_tapis_debug=True)

result


(
 active: True
 bucket: Mesonet2023-01-10T14:38:52.900824
 created_at: 2023-01-11 00:38:59.570629
 description: TEST project for MesoNet
 funding_resource: test
 last_updated: 2023-01-11 00:38:59.570646
 owner: testuser2
 permissions: 
 users: ['testuser2']
 pi: wongy
 project_id: Mesonet2023-01-10T14:38:52.900824
 project_name: Mesonet2023-01-10T14:38:52.900824
 project_url: https://www.hawaii.edu/climate-data-portal/,
 <tapipy.tapis.Debug at 0x1107552b0>)

### Creating the Site

In [12]:
## Create a Streams Site
result, debug = permitted_client.streams.create_site(project_id=project_id,
                                                     request_body=[{
                                                     "site_name":site_id, 
                                                     "site_id":site_id,
                                                     "latitude":50, 
                                                     "longitude":10, 
                                                     "elevation":2,
                                                     "description":'test_site'
                                                    }], _tapis_debug=True)
print(result)

[
chords_id: 773
created_at: 2023-01-11 00:39:06.399530
description: test_site
elevation: 2
last_updated: 2023-01-11 00:39:06.399576
latitude: 50
location: 
coordinates: [10.0, 50.0]
type: Point
longitude: 10
site_id: ParkHQ
site_name: ParkHQ]


### Creating the Instrument

In [15]:
result, debug = permitted_client.streams.create_instrument(project_id=project_id,
                                                           site_id=site_id,
                                                           request_body=[{
                                                            "inst_name":instrument_id,
                                                            "inst_description": "0151_ParkHQ",
                                                            "inst_id":instrument_id
                                                           }], _tapis_debug=True)

print(result)

[
chords_id: 770
created_at: 2023-01-11 00:39:43.909589
inst_description: 0151_ParkHQ
inst_id: 0151_ParkHQ2023-01-10T14-39-35-998980
inst_name: 0151_ParkHQ2023-01-10T14-39-35-998980
last_updated: 2023-01-11 00:39:43.909604]


### Parsing Variable Names from ikewai MesoNet file

In [16]:
import urllib
from datetime import date, timedelta, datetime
from dateutil.rrule import rrule, DAILY

# start_date = date(2023, 1, 1)
# end_date = datetime.today().date()

for dt in rrule(DAILY, dtstart=start_date, until=end_date):
    curr_date = dt.date()

    files = [
        "0115_Piiholo_MetData.dat",
        "0116_Keokea_MetData.dat",
        "0119_KulaAg_MetData.dat",
        "0143_Nakula_MetData.dat",
        "0151_ParkHQ_MetData.dat",
        "0152_NeneNest_MetData.dat",
        "0153_Summit_MetData.dat",
        "0281_IPIF_MetData.dat",
        "0282_Spencer_MetData.dat",
        "0283_Laupahoehoe_MetData.dat",
        "0286_Palamanui_MetData.dat",
        "0287_Mamalahoa_MetData.dat",
        "0501_Lyon_MetData_5min.dat",
        "0502_NuuanuRes1_MetData.dat",
        "0601_Waipa_MetData.dat",
        "0602_CommonGround_MetData.dat"
    ]
    base_url = "https://ikeauth.its.hawaii.edu/files/v2/download/public/system/ikewai-annotated-data/HCDP/raw/"

    year = curr_date.year
    month = curr_date.month
    day = curr_date.day

    for file in files:
        link = f"{base_url}{year}/{month}/{day}/{file}"

    data_file = urllib.request.urlopen(link).readlines()

    list_vars = data_file[1].decode("UTF-8").strip().replace("\"", "").split(",")

    list_units = data_file[2].decode("UTF-8").strip().replace("\"", "").split(",")
    

    
# link = 'https://ikeauth.its.hawaii.edu/files/v2/download/public/system/ikewai-annotated-data/HCDP/raw/2022/10/25/0151_ParkHQ_MetData.dat'

# data_file = urllib.request.urlopen(link).readlines()

# list_vars = data_file[1].decode("UTF-8").strip().replace("\"", "").split(",")

# list_units = data_file[2].decode("UTF-8").strip().replace("\"", "").split(",")

print(list_vars, list_units)

['TIMESTAMP', 'RECORD', 'dnWelling_SW_Avg', 'upWelling_SW_Avg', 'dnWelling_LW_Avg', 'upWelling_LW_Avg', 'Trnet_Avg', 'Rnet_Avg', 'SWnet_Avg', 'LWnet_Avg', 'TOT_dnWelling_Avg', 'TOT_upWelling_Avg', 'Albedo_Avg', 'Tsurface_Avg', 'Tsky_Avg', 'Tair_1_Avg', 'RH_1_Avg', 'Tair_2_Avg', 'RH_2_Avg', 'TC_air_Avg', 'WindSpeed', 'WindSpeed_rs', 'WindDir', 'WindDir_sd', 'WindDir_uv', 'Tsoil_Avg', 'SoilHeatFlux_Avg(1)', 'SoilHeatFlux_Avg(2)', 'F', 'M', 'G', 'SoilMoisture_Avg(1)', 'SoilMoisture_Avg(2)', 'SoilMoisture_T_Avg(1)', 'SoilMoisture_T_Avg(2)', 'FOG_Tot', 'Rainfall_Tot'] ['TS', 'RN', 'W/M^2', 'W/M^2', 'W/M^2', 'W/m^2', '', 'W/M^2', 'W/m^2', 'W/M^2', 'W/M^2', 'W/M^2', 'Ratio', 'Deg C', 'Deg C', 'Deg C', '%', 'Deg C', '%', 'Deg C', 'm/s', 'm/s', 'Deg', 'Deg', 'Deg', 'Deg C', 'W/m^2', 'W/m^2', '', '', 'W/M^2', 'm^3/m^3', 'm^3/m^3', 'm^3/m^3', 'm^3/m^3', 'counts', 'mm']


### Creating the Variables

In [19]:

variables = []

for i in range(4, len(data_file)):
    measurements = data_file[i].decode("UTF-8").strip().replace("\"", "").split(",")
    measurement = {}
    time = measurements[0].split(" ")

    if(int(time[1].split(":")[0]) > 23):
        time_string = time[0] + " 23:59:59"
        time_string = datetime.strptime(time_string, '%Y-%m-%d %H:%M:%S')
        time_string += timedelta(seconds=1)
    else:
        time_string = datetime.strptime(measurements[0], '%Y-%m-%d %H:%M:%S')

    measurement['datetime'] = time_string.isoformat()+"-10:00"
    for j in range(2, len(measurements)):
        measurement[list_vars[j]] = float(measurements[j])
    variables.append(measurement)

result = permitted_client.streams.create_measurement(inst_id=instrument_id,
                                                     vars=variables)

print(result)


Albedo_Avg: [
2022-10-25T00:05:00-10:00: nan, 
2022-10-25T00:10:00-10:00: nan, 
2022-10-25T00:15:00-10:00: nan, 
2022-10-25T00:20:00-10:00: nan, 
2022-10-25T00:25:00-10:00: nan, 
2022-10-25T00:30:00-10:00: nan, 
2022-10-25T00:35:00-10:00: nan, 
2022-10-25T00:40:00-10:00: nan, 
2022-10-25T00:45:00-10:00: nan, 
2022-10-25T00:50:00-10:00: nan, 
2022-10-25T00:55:00-10:00: nan, 
2022-10-25T01:00:00-10:00: nan, 
2022-10-25T01:05:00-10:00: nan, 
2022-10-25T01:10:00-10:00: nan, 
2022-10-25T01:15:00-10:00: nan, 
2022-10-25T01:20:00-10:00: nan, 
2022-10-25T01:25:00-10:00: nan, 
2022-10-25T01:30:00-10:00: nan, 
2022-10-25T01:35:00-10:00: nan, 
2022-10-25T01:40:00-10:00: nan, 
2022-10-25T01:45:00-10:00: nan, 
2022-10-25T01:50:00-10:00: nan, 
2022-10-25T01:55:00-10:00: nan, 
2022-10-25T02:00:00-10:00: nan, 
2022-10-25T02:05:00-10:00: nan, 
2022-10-25T02:10:00-10:00: nan, 
2022-10-25T02:15:00-10:00: nan, 
2022-10-25T02:20:00-10:00: nan, 
2022-10-25T02:25:00-10:00: nan, 
2022-10-25T02:30:00-10:00: n

In [17]:
request_body = []
for i in range(2, len(list_vars)):
    request_body.append({
        "var_id": list_vars[i],
        "var_name": list_vars[i],
        "units": list_units[i]
    })

#Create variables in bulk
result, debug = permitted_client.streams.create_variable(project_id=project_id,
                                                         site_id=site_id,
                                                         inst_id=instrument_id,
                                                         request_body=request_body,_tapis_debug=True)

print(result)

[
chords_id: 899
created_at: 2023-01-11 00:39:59.341056
last_updated: 2023-01-11 00:39:59.341076
units: W/M^2
var_id: dnWelling_SW_Avg
var_name: dnWelling_SW_Avg, 
chords_id: 900
created_at: 2023-01-11 00:39:59.573936
last_updated: 2023-01-11 00:39:59.573957
units: W/M^2
var_id: upWelling_SW_Avg
var_name: upWelling_SW_Avg, 
chords_id: 901
created_at: 2023-01-11 00:39:59.780560
last_updated: 2023-01-11 00:39:59.780580
units: W/M^2
var_id: dnWelling_LW_Avg
var_name: dnWelling_LW_Avg, 
chords_id: 902
created_at: 2023-01-11 00:39:59.987854
last_updated: 2023-01-11 00:39:59.987866
units: W/m^2
var_id: upWelling_LW_Avg
var_name: upWelling_LW_Avg, 
chords_id: 903
created_at: 2023-01-11 00:40:00.194714
last_updated: 2023-01-11 00:40:00.194734
units: 
var_id: Trnet_Avg
var_name: Trnet_Avg, 
chords_id: 904
created_at: 2023-01-11 00:40:00.396448
last_updated: 2023-01-11 00:40:00.396461
units: W/M^2
var_id: Rnet_Avg
var_name: Rnet_Avg, 
chords_id: 905
created_at: 2023-01-11 00:40:00.603695
last_up

### Parsing Measurements from MesoNet file